# CODE FOR CREATING THE DATASET 

Once we have downloaded the mel spectrograms from the computer terminal, the next step is to create the dataset by extracting specific features from each of the downloaded mels and adding their corresponding tags. We now have 35 folders containing mel spectrograms, with around 200 in each. In total, we obtained 6,913 observations to be added to the dataset. 

I would like to mention that in this Python notebook, the only thing you (the reader) will need to change for the code to run are the file paths, such as "C:/Users/PORTATIL/Desktop/mtg_data/melspecs_extraidos". This path only works on my computer, but if you replace it with the folder where your data is stored, the code will work on yours as well.

## LIBRARIES

In [ ]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import librosa

## MEL SPECTOGRAM FEATURES

#### 00 FOLDER

The initial idea was to extract the following features: tempo, onsets per second, time signature, MFCCs, spectral centroid, spectral rolloff, key, root mean square (RMS), loudness, dynamic range, richness, and novelty. However, so far we have only downloaded the Mel spectrograms, and we can’t extract all the features we initially planned because some of them (tempo/BPM, time signature, onsets, average loudness, dynamic range, and key) require access to the original audio waveform for proper analysis, and we only have the .npy files. Since Mel spectrograms don’t retain information about phase, absolute amplitude, or fine temporal structure, those features can’t be derived from the data we currently have.

Therefore, we’ve decided to replace those features with others that are compatible with the information available from the Mel spectrograms. The final list looks as follows:

* **MFCCs** → Mel-frequency cepstral coefficients. Capture the general “timbre” of the sound.
* **Spectral centroid** → Indicates whether the energy is concentrated in higher (brighter) or lower (darker) frequencies.
* **Spectral rolloff** → The frequency below which a certain percentage of the total spectral energy is contained.
* **Spectral richness (Frequency variance)** → The variability of energy across frequency bands, indicating timbral complexity.
* **Energy per frame** → Total energy of each column in the spectrogram (i.e., intensity at each moment in time).
* **Variance of energy** → How much the energy changes over time (variation in intensity).
* **Spectral flux** → Measures how much the energy distribution changes from one frame to the next. Useful for detecting onsets and section changes.
* **Temporal centroid** → The point in time where most of the energy is concentrated (e.g., beginning, middle, or end of the segment).
* **RMS per frame** → Root mean square energy per frame. Roughly corresponds to perceived loudness at each moment.
* **Spectral novelty** → Measures how much the Mel matrix changes from one frame to the next. Useful for detecting significant musical transitions.
* **Attack/Decay slope** → The rate of change in energy at the start and end of a musical event, serving as an indicator of temporal dynamics.


We will start by analyzing a single folder, just to look at the information we have and to have an idea of what we want to do. 

In [ ]:

# Define the folder from which to read the .npy files
carpeta = "C:/Users/PORTATIL/Desktop/mtg_data/melspecs_extraidos/00"

# Create a list for keeping the data
datos = []
archivos = [f for f in os.listdir(carpeta) if f.endswith(".npy")]

# Read each .npy
for archivo in tqdm(archivos, desc=f"Procesando carpeta 00"):
    ruta = os.path.join(carpeta, archivo)
    mel_array = np.load(ruta)           
    datos.append(["00", archivo, mel_array]) 

# Create dataframe
df = pd.DataFrame(datos, columns=["carpeta", "archivo", "mel_array"])

# See results
print(df.head())


We obtain a table that shows the source folder of each file, the file name, and the corresponding *mel\_array*. The *mel\_array* represents the spectrogram in the form of a 2D matrix, where rows correspond to frequency components and columns represent time frames.
Now that we have seen how to analyze the data from a single folder, we will now extend the process to all folders and extract the previously mentioned features.
 


In [ ]:

# Extract features from mel spectograms
def obtener_features_mel(mel_array, sr=22050, hop_length=512):
    
    # Convert from dB to amplitude
    mel_array = librosa.db_to_amplitude(mel_array)

    # MFCCs
    mfccs = librosa.feature.mfcc(S=librosa.power_to_db(mel_array), sr=sr)
    mfcc_mean = np.mean(mfccs, axis=1)

    # Centroid
    centroid = librosa.feature.spectral_centroid(S=mel_array, sr=sr)
    centroid_mean = np.mean(centroid)

    # Rolloff 
    rolloff = librosa.feature.spectral_rolloff(S=mel_array, sr=sr)
    rolloff_mean = np.mean(rolloff)

    # Richness
    richness = np.std(mel_array, axis=1).mean()

    # Energy per frame
    energy_per_frame = np.sum(mel_array, axis=0)
    var_energy = np.var(energy_per_frame)

    # Spectral flux and novelty
    flux = np.sqrt(np.sum(np.diff(mel_array, axis=1)**2, axis=0))
    flux_mean = np.mean(flux)
    novelty = flux_mean

    # Temporal centroid
    times = librosa.frames_to_time(np.arange(mel_array.shape[1]), sr=sr, hop_length=hop_length)
    temporal_centroid = np.sum(times * energy_per_frame) / np.sum(energy_per_frame)

    # RMS and attack/decay slope
    rms = np.sqrt(np.mean(np.square(mel_array), axis=0))
    rms_mean = np.mean(rms)
    attack_decay_slope = np.mean(np.abs(np.gradient(rms)))

    # Build feature dictionary
    features = {
        **{f"mfcc_{i}": mfcc_mean[i] for i in range(len(mfcc_mean))},
        "centroid": centroid_mean,
        "rolloff": rolloff_mean,
        "richness": richness,
        "var_energy": var_energy,
        "spectral_flux": flux_mean,
        "temporal_centroid": temporal_centroid,
        "rms": rms_mean,
        "novelty": novelty,
        "attack_decay_slope": attack_decay_slope,
    }

    return features

# Process all .npy files in a folder, extracts features, and saves them to a CSV file
def procesar_carpeta(carpeta_dir, carpeta_salida, sr=22050, hop_length=512):
    
    datos = []
    carpeta_nombre = os.path.basename(carpeta_dir)
    archivos = [f for f in os.listdir(carpeta_dir) if f.endswith('.npy')]

    for archivo in tqdm(archivos, desc=f"Procesando carpeta: {carpeta_nombre}"):
        ruta = os.path.join(carpeta_dir, archivo)
        try:
            mel_array = np.load(ruta)
            features = obtener_features_mel(mel_array, sr=sr, hop_length=hop_length)
            track_id = os.path.splitext(archivo)[0]

            features["track_id"] = track_id
            features["carpeta"] = carpeta_nombre
            datos.append(features)

        except Exception as e:
            print(f" Error processing {archivo}: {e}")

    # Create DataFrame
    df = pd.DataFrame(datos)

    # Reorder columns (track\_id and folder first)
    columnas_ordenadas = ['track_id', 'carpeta'] + [col for col in df.columns if col not in ['track_id', 'carpeta']]
    df = df[columnas_ordenadas]
    df.set_index("track_id", inplace=True)

    # Save CSV
    os.makedirs(carpeta_salida, exist_ok=True)
    ruta_salida = os.path.join(carpeta_salida, f"features_clean_{carpeta_nombre}.csv")
    df.to_csv(ruta_salida)
    print(f"File saved in: {ruta_salida}")

    return df

# Use of the script
if __name__ == "__main__":
    carpeta_mel = "C:/Users/PORTATIL/Desktop/mtg_data/melspecs_extraidos/00"
    carpeta_salida = "C:/Users/PORTATIL/Desktop/mtg_data/data_limpia"

    df_features = procesar_carpeta(carpeta_mel, carpeta_salida)
    print(df_features.head())


We will conduct an analysis of the first 2 files:

**1009600.npy** → This file presents a distribution of MFCCs where MFCC\_0 is quite low (-181.78), suggesting that this track may have a relatively lower overall energy compared to others. The spectral centroid (2439.58 Hz) indicates that the energy is distributed in the mid-frequency range, while the spectral richness (0.3257) and energy variance (173.66) point to a moderate variety of frequency components which is not particularly high. Both the spectral flux (1.33) and novelty (1.33) are moderate, which may imply that this piece does not exhibit abrupt spectral changes, especially when compared to more intense tracks. The RMS value (0.48) reflects a rather restrained average intensity.

**1012000.npy** → This file shows a less extreme MFCC\_0 value (-120.22), which suggests a higher overall energy level than the first file. The spectral centroid (1514.70 Hz) is lower, indicating that the track leans toward a less bright sound, with more energy concentrated in the low and mid frequencies. The energy variance (108.48) and spectral richness (0.214) are somewhat lower than in the previous example, implying less variability and a simpler spectral texture. However, the spectral flux and novelty values (3.13) are higher, which could suggest that this piece exhibits more dynamic spectral changes and a more pronounced temporal evolution.

In summary,  from our function we obtain the following:

* **mfcc\_0–mfcc\_19**: Capture the general timbre of the sound.
* **Centroid, rolloff, richness**: Describe the distribution of energy across frequencies.
* **var\_energy, spectral\_flux, temporal\_centroid**: Capture the temporal evolution of energy.
* **rms, novelty, attack\_decay\_slope**: Reflect aspects of intensity and dynamic behavior.

Let’s clarify why MFCCs range from 0 to 19. MFCC stands for *Mel-Frequency Cepstral Coefficients*. These coefficients are used to represent the general shape of the audio spectrum on a scale that aligns with human hearing. When computing MFCCs, the output is a matrix where **MFCC\_0** captures the overall energy of the spectrum (general intensity level), while **MFCC\_1 to MFCC\_19** represent increasingly fine details of the spectral distribution; such as brightness or darkness, frequency profile, and timbral texture. The use of 20 coefficients (from 0 to 19) is standard in audio analysis, as this number is typically sufficient to represent most musical timbres. 

With these features in place, the goal is to apply the same extraction process to the remaining folders using a for loop (for avoiding overloading the computer and preventing memory errors) and then merge everything into a single dataframe.

#### ALL FOLDERS

In [ ]:

# Function for obtaining features 
def obtener_features_mel(mel_array, sr=22050, hop_length=512):
    mel_array = librosa.db_to_amplitude(mel_array)

    mfccs = librosa.feature.mfcc(S=librosa.power_to_db(mel_array), sr=sr)
    mfcc_mean = np.mean(mfccs, axis=1)

    centroid = librosa.feature.spectral_centroid(S=mel_array, sr=sr).flatten()
    rolloff = librosa.feature.spectral_rolloff(S=mel_array, sr=sr).flatten()
    flux = np.sqrt(np.sum(np.diff(mel_array, axis=1)**2, axis=0))
    rms = np.sqrt(np.mean(mel_array**2, axis=0))
    energy_per_frame = np.sum(mel_array, axis=0)

    features = {
        **{f"mfcc_{i}": mfcc_mean[i] for i in range(len(mfcc_mean))},
        "centroid": np.mean(centroid),
        "rolloff": np.mean(rolloff),
        "richness": np.std(mel_array, axis=1).mean(),
        "var_energy": np.var(energy_per_frame),
        "spectral_flux": np.mean(flux),
        "temporal_centroid": np.sum(librosa.frames_to_time(np.arange(mel_array.shape[1]), sr=sr, hop_length=hop_length) * energy_per_frame) / np.sum(energy_per_frame),
        "rms": np.mean(rms),
        "novelty": np.mean(flux),
        "attack_decay_slope": np.mean(np.abs(np.gradient(rms))),
    }

    return features

# Function for processing a folder
def procesar_carpeta(carpeta_dir, nombre_carpeta, output_dir):
    archivos = [f for f in os.listdir(carpeta_dir) if f.endswith('.npy')]
    datos = []

    for archivo in tqdm(archivos, desc=f" Processing folder {nombre_carpeta}"):
        ruta = os.path.join(carpeta_dir, archivo)
        try:
            mel_array = np.load(ruta)
            features = obtener_features_mel(mel_array)
            track_id = os.path.splitext(archivo)[0]
            fila = {"track_id": track_id, "carpeta": nombre_carpeta, **features}
            datos.append(fila)
        except Exception as e:
            print(f" Error in file {archivo}: {e}")

    df = pd.DataFrame(datos)
    df = df[["track_id", "carpeta"] + [col for col in df.columns if col not in ["track_id", "carpeta"]]]
    
    os.makedirs(output_dir, exist_ok=True)
    ruta_salida = os.path.join(output_dir, f"features_clean_{nombre_carpeta}.csv")
    df.to_csv(ruta_salida, index=False)
    print(f" Saved: {ruta_salida}")
    return df

# Process in chunks of 5 folders
carpeta_base = "C:/Users/PORTATIL/Desktop/mtg_data/melspecs_extraidos"
output_dir = "C:/Users/PORTATIL/Desktop/mtg_data/features_limpios"

carpetas = [f"{i:02d}" for i in range(36)]  # 00 a 35
chunk_size = 5

for i in range(0, len(carpetas), chunk_size):
    bloque = carpetas[i:i+chunk_size]
    print(f"\n Processing folder chunk: {bloque}")

    for carpeta in bloque:
        carpeta_dir = os.path.join(carpeta_base, carpeta)
        if os.path.isdir(carpeta_dir):
            procesar_carpeta(carpeta_dir, carpeta, output_dir)
        else:
            print(f" Folder {carpeta_dir} not found")


Let’s check the head of the DataFrame from the first 5 folders to see if everything was processed as expected.

In [ ]:

# Path for obtaining cleaned CSV files
ruta_limpios = "C:/Users/PORTATIL/Desktop/mtg_data/features_limpios"

# Ordered list of the first 5 CSV files
csv_files = sorted([f for f in os.listdir(ruta_limpios) if f.endswith(".csv")])[:5]

# Show head of each file
for archivo in csv_files:
    ruta = os.path.join(ruta_limpios, archivo)
    print(f"\n Showing head of: {archivo}")
    df = pd.read_csv(ruta)
    print(df.head())

#### SAVING FULL DATASET

In [ ]:
# Path of cleaned CSV files
ruta_limpios = "C:/Users/PORTATIL/Desktop/mtg_data/features_limpios"

# List of all generated CSV files
csv_files = sorted([f for f in os.listdir(ruta_limpios) if f.endswith(".csv")])

# Read and merge all the files
df_final = pd.concat(
    [pd.read_csv(os.path.join(ruta_limpios, archivo)) for archivo in csv_files],
    ignore_index=True
)

# Summary of final dataframe
print("Final dataset created with shape:", df_final.shape)
print(df_final.head())

# Save as CSV
ruta_salida_csv = "C:/Users/PORTATIL/Desktop/mtg_data/features_final.csv"
df_final.to_csv(ruta_salida_csv, index=False)
print(f" Saved in: {ruta_salida_csv}")


We now have the dataset with the extracted features. The next step is to merge it with the mood/theme metadata by adding a new column that indicates the mood/theme associated with each track. We'll perform the merge using the track_id as the common key between both datasets.

## MOOD/THEME

It is necessary to examine the structure of the mood/theme file in order to understand its format and determine the most appropriate way to process and integrate it with the feature dataset.

In [ ]:
# Path to the mood/theme file
ruta_mood = "C:/Users/PORTATIL/Desktop/Carpetas R/TFM/mtg-jamendo-dataset-master/mtg-jamendo-dataset-master/data/autotagging_moodtheme.tsv"

# Show structure
with open(ruta_mood, encoding='utf-8') as f:
    for _ in range(10):
        print(f.readline())

As observed, the document contains a **track_id** column and a **tags** column (along with others). The track_id column will be used to merge this information with our dataset. In the following code block, the tags column is handled in a special way because each track can have multiple mood/theme tags separated by tab characters within the same line of the file. Instead of splitting each tag into separate columns from the start (which could break the DataFrame structure if the number of tags varies), all tags are grouped into a single column as a list of strings. This step is essential for preserving the complete and flexible information, as it later allows us to safely split these lists of tags into individual columns (tag1, tag2, etc.) in a structured manner.


In [ ]:
ruta_mood = "C:/Users/PORTATIL/Desktop/Carpetas R/TFM/mtg-jamendo-dataset-master/mtg-jamendo-dataset-master/data/autotagging_moodtheme.tsv"

# Lists to store data
rows = []

with open(ruta_mood, encoding='utf-8') as f:
    header = f.readline().strip().split('\t')
    
    for line in f:
        parts = line.strip().split('\t')
        base_fields = parts[:5]
        tags = parts[5:]
        rows.append(base_fields + [tags])

# Create dataframe with column names
columnas = header[:5] + ['TAGS']
df_mood = pd.DataFrame(rows, columns=columnas)

# Extract numeric track_id 
df_mood['track_id'] = df_mood['TRACK_ID'].str.extract(r'(\d+)$').astype(int)

print(df_mood.head())

We can now see that the tags column is a list of strings, and that we have generated a new track_id containing only the numeric identifier. Now, we are ready to separate the tags column into different tags. 


In [ ]:
# Expand list of tags into separate columns
df_tags_expandido = df_mood['TAGS'].apply(pd.Series)
df_tags_expandido.columns = [f'tag{i+1}' for i in range(df_tags_expandido.shape[1])]

# Concatenate the expanded tags as new columns to the original dataframe
df_mood_expandido = pd.concat([df_mood.drop(columns=['TAGS']), df_tags_expandido], axis=1)

print(df_mood_expandido)

The tracks with multiple tags now have those tags separated into individual columns. Let’s proceed to merge this with our final dataset.


In [ ]:
df_features['track_id'] = df_features['track_id'].astype(int)
df_mood_expandido['track_id'] = df_mood_expandido['track_id'].astype(int)

# Merge
df_completo = pd.merge(df_features, df_mood_expandido, on='track_id', how='left')

print(df_completo.head())


We can see that all columns have been merged, not just the tags and track_id. We'll remove the unnecessary ones. Additionally, we want the tags to display only the label names, removing the prefix **mood/theme---**.

First, let's check the names of all columns in the current dataset to decide which ones to drop.


In [ ]:
print(df_completo.columns.tolist())

These are the columns we need to remove: "TRACK_ID", "ARTIST_ID", "ALBUM_ID", "PATH", "DURATION".


In [ ]:
# Remove columns
columnas_a_eliminar = ['TRACK_ID', 'ARTIST_ID', 'ALBUM_ID', 'PATH', 'DURATION']
df_completo = df_completo.drop(columns=[col for col in columnas_a_eliminar if col in df_completo.columns])

print(df_completo.head(15))

Last thing is to remove the prefix mood/theme---

In [ ]:
# Identify tag columns 
columnas_tag = [col for col in df_completo.columns if col.startswith('tag')]

# Replace 'mood/theme---' 
for col in columnas_tag:
    df_completo[col] = df_completo[col].map(
        lambda x: x.replace('mood/theme---', '') if isinstance(x, str) else x
    )

# Results
print(df_completo.head())

We will save the dataset 

In [ ]:
ruta_salida = "C:/Users/PORTATIL/Desktop/mtg_data/features_final_mood_theme.csv"
df_completo.to_csv(ruta_salida, index=False)

print("Saved in:", ruta_salida)

## ADDING MP3s

We analyzed the features extracted from the mel spectrograms, but they are not sufficient for the scale of the task we want to perform. Therefore, we decided to download the MP3 audio files in order to extract additional features from them. We will begin processing them and attempting to merge this information with the existing table containing the Mel spectrograms.

As a first step, we will simply try to read the MP3 files and create a table that contains the name of each file along with the folder it comes from.


In [ ]:
# Path where the downloaded MP3 files are located
base_path = r"E:\TFM\mp3_low"

# List for saving data
data = []

# Analyze all the folders and files
for root, _, files in os.walk(base_path):
    for file in files:
        if file.endswith(".mp3"):
            full_path = os.path.join(root, file)
            folder_name = os.path.basename(root)
            data.append({
                "archivo": file,
                "carpeta_origen": folder_name
            })

# Create DataFrame
df = pd.DataFrame(data)
 
print(df)

We obtained what we wanted. We are now going to remove the **.low\.mp3** suffix from each file name so that we can later merge it with our other table.


In [ ]:
df["archivo"] = df["archivo"].str.replace(".low.mp3", "", regex=False)

print(df.head())

We will also select only the tracks that have both a Mel spectrogram and an MP3 file, so we can work more efficiently and comfortably with the MP3 data.


In [ ]:
# Read csv
csv_path = r"C:\Users\PORTATIL\Desktop\mtg_data\features_final_mood_theme.csv"
df_ids = pd.read_csv(csv_path)

# Make sure that type of data is string
df_ids["track_id"] = df_ids["track_id"].astype(str)

# Convert "archivo" column into string 
df["archivo"] = df["archivo"].astype(str)

# Filter tracks with both mel and mp3
df_filtrado = df[df["archivo"].isin(df_ids["track_id"])]

# See number of coincidences
print(f"Coincidences: {len(df_filtrado)}")

print(df_filtrado.head())

We can see that there are approximately 7,000 files. From those we have retained we are going to extract a set of features in order to later merge them with our existing table and build a more complete dataset.

The features to be extracted from the MP3 audio files are:
- Tempo / BPM → The overall speed of the song, measured in beats per minute. Calculated using beat detection from the audio waveform.
- Onsets per second → Frequency of distinct sonic events (e.g., drum hits or note attacks) per second.
- RMS → Root Mean Square energy; indicates the average intensity of the track.
- RMS Dynamic Range → Difference between the loudest and softest moments in the track, reflecting its dynamic expressiveness.
- Loudness → Estimated average volume in decibels; reflects how loud the track sounds overall.
- Temporal Centroid → The approximate moment in the track where most of the energy is concentrated.
- Key Estimate → The estimated musical key (0 = C, 1 = C♯, ..., 11 = B); represents the tonal center of the piece.

We  will proceed in a way that avoids overloading the system when processing all 7,000 audio files. We will divide the data into chunks of 500 files and process each block separately. After processing all blocks, we will combine the results into a final dataset.





In [ ]:
# Read the 7000 files
df = pd.read_csv("mp3_filtrados.csv")

# Base path where the files are organized by folders
base_path = r"E:\TFM\mp3_low"

# List for saving results
all_features = []

# Chunk size
chunk_size = 500
num_chunks = (len(df) // chunk_size) + 1

print(f"Processing {len(df)} files in {num_chunks} chunks...")

# Process by chunk
for chunk_idx in range(num_chunks):
    print(f"Processing chunk {chunk_idx + 1}/{num_chunks}...")
    chunk = df.iloc[chunk_idx * chunk_size : (chunk_idx + 1) * chunk_size]

    for _, row in tqdm(chunk.iterrows(), total=len(chunk), leave=False):
        track_id = str(row["archivo"])
        folder = str(row["carpeta_origen"]).zfill(2)
        file_path = os.path.join(base_path, folder, f"{track_id}.low.mp3")

        if not os.path.exists(file_path):
            continue

        try:
            y, sr = librosa.load(file_path, sr=None, duration=30)

            # Tempo
            tempo, _ = librosa.beat.beat_track(y=y, sr=sr)

            # Onsets per second
            onsets = librosa.onset.onset_detect(y=y, sr=sr)
            duration = librosa.get_duration(y=y, sr=sr)
            onsets_per_sec = len(onsets) / duration if duration > 0 else 0

            # RMS and dynamic range
            rms = librosa.feature.rms(y=y)[0]
            rms_mean = np.mean(rms)
            rms_dynamic_range = np.max(rms) - np.min(rms)

            # Loudness
            loudness = librosa.amplitude_to_db(np.abs(y), ref=np.max)
            loudness_mean = np.mean(loudness)

            # Temporal centroid
            times = np.arange(len(rms))
            temporal_centroid = np.sum(times * rms) / np.sum(rms) if np.sum(rms) > 0 else 0

            # Key
            chroma = librosa.feature.chroma_stft(y=y, sr=sr)
            key_estimate = np.argmax(np.mean(chroma, axis=1))

            # Append to the main list
            all_features.append({
                "track_id": track_id,
                "tempo_bpm": tempo,
                "onsets_per_sec": onsets_per_sec,
                "rms_mean": rms_mean,
                "rms_dynamic_range": rms_dynamic_range,
                "loudness_mean_db": loudness_mean,
                "temporal_centroid": temporal_centroid,
                "key_estimate": key_estimate
            })

        except Exception as e:
            print(f"Error in {track_id}: {e}")
            continue

# Save results
features_df = pd.DataFrame(all_features)
features_df.to_csv("mp3_features.csv", index=False)
print("Extraction completed. Saved as 'mp3_features.csv'")

features_df


We now have the dataset ready. Let’s proceed to merge the two tables: the one containing features from the MP3 files and the one with features from the Mel spectrograms.

As mentioned in the beginning, we will use the track_id as the common key to align the rows correctly.

In [ ]:
# Load Mel spectrogram table
mel_df = pd.read_csv(r"C:\Users\PORTATIL\Desktop\mtg_data\features_final_mood_theme.csv")

# Load mp3 table
audio_df = pd.read_csv("mp3_features.csv")

# Ensure that IDs are strings
mel_df["track_id"] = mel_df["track_id"].astype(str)
audio_df["track_id"] = audio_df["track_id"].astype(str)

# Merge tables (by track_id)
merged_df = mel_df.merge(audio_df, on="track_id", how="inner")

# Drop 'carpeta\_origen' if it was mistakenly included from another table
if "carpeta_origen" in merged_df.columns:
    merged_df = merged_df.drop(columns=["carpeta_origen"])

# Save dataset
merged_df.to_csv("tabla_completa_unificada.csv", index=False)
print("Final table saved as 'tabla_completa_unificada.csv'")

merged_df

## INSTRUMENTS

To enhance our exploratory data analysis (or just to have another view of it in case of being necessary), we will also include metadata about the instruments present in each track and the genre to which each song is classified.

We will start with the instruments. Let's begin by examining the structure of the document to understand how the information is organized and how we can process it.

In [ ]:

with open("C:/Users/PORTATIL/Desktop/Carpetas R/TFM/mtg-jamendo-dataset-master/mtg-jamendo-dataset-master/data/autotagging_instrument.tsv", encoding='utf-8') as f:
    for i in range(30): 
        print(f.readline())

As we can see, the document has the same structure as the mood/theme file, except that this time the tags refer to instruments. Therefore, we will follow the same process to incorporate them into our dataset. 

We will not provide as many explanations as for the mood tags, since it is understood that they have already been given previously and everything is clear, as we are simply copying the process.


In [ ]:
ruta_inst = "C:/Users/PORTATIL/Desktop/Carpetas R/TFM/mtg-jamendo-dataset-master/mtg-jamendo-dataset-master/data/autotagging_instrument.tsv"

# Lists to store data
rows = []

with open(ruta_inst, encoding='utf-8') as f:
    header = f.readline().strip().split('\t')
    
    for line in f:
        parts = line.strip().split('\t')
        base_fields = parts[:5]
        tags = parts[5:]
        rows.append(base_fields + [tags])

# Create dataframe with column names
columnas = header[:5] + ['TAGS']
df_inst = pd.DataFrame(rows, columns=columnas)

# Extract numeric track_id 
df_inst['track_id'] = df_inst['TRACK_ID'].str.extract(r'(\d+)$').astype(int)

print(df_inst.head())

In [ ]:
# Expand list of tags into separate columns
df_inst_expandido = df_inst['TAGS'].apply(pd.Series)
df_inst_expandido.columns = [f'inst{i+1}' for i in range(df_inst_expandido.shape[1])]

# Merge with original dataframe
df_inst_expandido = pd.concat([df_inst.drop(columns=['TAGS']), df_inst_expandido], axis=1)

print(df_inst_expandido)

In [ ]:
merged_df = pd.read_csv(r"C:\Users\PORTATIL\Desktop\mtg_data\tabla_completa_unificada.csv")

merged_df['track_id'] = merged_df['track_id'].astype(int)
df_inst_expandido['track_id'] = df_inst_expandido['track_id'].astype(int)

# Merge
df_mood_inst = pd.merge(merged_df, df_inst_expandido, on='track_id', how='left')

print(df_mood_inst.head())

In [ ]:
# Check column names
print(df_mood_inst.columns.tolist())

In [ ]:
# Remove unnecessary columns
columnas_quitar = ['TRACK_ID', 'ARTIST_ID', 'ALBUM_ID', 'PATH', 'DURATION']
df_mood_inst = df_mood_inst.drop(columns=[col for col in columnas_quitar if col in df_mood_inst.columns])

print(df_mood_inst.head(5))

df_mood_inst

In [ ]:
# Check new changes
print(df_mood_inst.columns.tolist())

In [ ]:
# Identify tag columns 
columnas_inst = [col for col in df_mood_inst.columns if col.startswith('inst')]

# Replace 'instrument---' 
for col in columnas_inst:
    df_mood_inst[col] = df_mood_inst[col].map(
        lambda x: x.replace('instrument---', '') if isinstance(x, str) else x
    )

# Results
print(df_mood_inst.head())

In [ ]:
# Show all columns without truncating
pd.set_option('display.max_columns', None)

# Show first 5 rows of full DataFrame
print(df_mood_inst.head())

Perfect, what we did here is the same as with mood/theme, and it turned out just as we wanted. Now we not only have the features and mood/theme columns, but we have also added the instrument columns in the same format. Next, we are going to follow the exact same process to add the genre.

## GENRE

Let's begin by examining the structure of the document to understand how the information is organized and how we can process it.

In [ ]:
with open("C:/Users/PORTATIL/Desktop/Carpetas R/TFM/mtg-jamendo-dataset-master/mtg-jamendo-dataset-master/data/autotagging_genre.tsv", encoding='utf-8') as f:
    for i in range(30):  # muestra primeras 30 líneas
        print(f.readline())

We can see that the structure is exactly the same as before. Since the process is exactly the same as the one we followed in the mood/theme and instruments sections, we will not provide further explanations beyond commenting on the code in the chunks, so that it does not become tedious.

In [ ]:
ruta_genre = "C:/Users/PORTATIL/Desktop/Carpetas R/TFM/mtg-jamendo-dataset-master/mtg-jamendo-dataset-master/data/autotagging_genre.tsv"

# Lists to store data
rows = []

with open(ruta_genre, encoding='utf-8') as f:
    header = f.readline().strip().split('\t')
    
    for line in f:
        parts = line.strip().split('\t')
        base_fields = parts[:5]
        tags = parts[5:]
        rows.append(base_fields + [tags])

# Create dataframe with column names
columnas = header[:5] + ['TAGS']
df_genre = pd.DataFrame(rows, columns=columnas)

# Extract numeric track_id 
df_genre['track_id'] = df_genre['TRACK_ID'].str.extract(r'(\d+)$').astype(int)

print(df_genre.head())

In [ ]:
# Expand list of tags into separate columns
df_genre_expandido = df_genre['TAGS'].apply(pd.Series)
df_genre_expandido.columns = [f'genre{i+1}' for i in range(df_genre_expandido.shape[1])]

# Merge with original dataframe
df_genre_expandido = pd.concat([df_genre.drop(columns=['TAGS']), df_genre_expandido], axis=1)

print(df_genre_expandido)

In [ ]:
df_mood_inst['track_id'] = df_mood_inst['track_id'].astype(int)
df_genre_expandido['track_id'] = df_genre_expandido['track_id'].astype(int)

# Merge
df_mood_inst_genre = pd.merge(df_mood_inst, df_genre_expandido, on='track_id', how='left')

print(df_mood_inst_genre.head())

In [ ]:
# Check column names
print(df_mood_inst_genre.columns.tolist())

In [ ]:
# Remove columns
columnas_borrar = ['TRACK_ID', 'ARTIST_ID', 'ALBUM_ID', 'PATH', 'DURATION']
df_mood_inst_genre = df_mood_inst_genre.drop(columns=[col for col in columnas_borrar if col in df_mood_inst_genre.columns])

print(df_mood_inst_genre.head(5))


In [ ]:
# Identify tag columns 
columnas_genre = [col for col in df_mood_inst_genre.columns if col.startswith('genre')]

# Replace 'genre---' 
for col in columnas_genre:
    df_mood_inst_genre[col] = df_mood_inst_genre[col].map(
        lambda x: x.replace('genre---', '') if isinstance(x, str) else x
    )

# Results
print(df_mood_inst_genre.head())

In [ ]:
# Merge to dataset
ruta_salida = "C:/Users/PORTATIL/Desktop/mtg_data/df_mood_inst_genre.csv"
df_mood_inst_genre.to_csv(ruta_salida, index=False)

print("Saved in:", ruta_salida)

We now have the complete table with all the features and the tags for mood/theme, instruments, and genre. We can start analyzing the data and preparing it for the EDA in R.

## ADD DURATION FOR EDA

As we analyzed the data, we realized that we need to normalize some variables, and for that we require the duration of each track; therefore, we are going to extract it and add it to the dataset.

In the previous steps, we saw that the mood/theme, instruments, and genre documents included the duration of each track (which we removed without realizing we would need it). Therefore, we are going to use the same mood/theme document to add the duration back into the dataset.

In [ ]:
# Load current dataset (without duration)
df_actual = pd.read_csv(r"C:\Users\PORTATIL\Desktop\mtg_data\df_mood_inst_genre.csv")

# Load .tsv file that contains duration

ruta_mood = "C:/Users/PORTATIL/Desktop/Carpetas R/TFM/mtg-jamendo-dataset-master/mtg-jamendo-dataset-master/data/autotagging_moodtheme.tsv"

# Lists to store data
rows = []

with open(ruta_mood, encoding='utf-8') as f:
    header = f.readline().strip().split('\t')
    
    for line in f:
        parts = line.strip().split('\t')
        base_fields = parts[:5]
        tags = parts[5:]
        rows.append(base_fields + [tags])

# Create dataframe with column names
columnas = header[:5] + ['TAGS']
df_mood = pd.DataFrame(rows, columns=columnas)

# Extract numeric track_id 
df_mood['track_id'] = df_mood['TRACK_ID'].str.extract(r'(\d+)$').astype(int)

print(df_mood.head())

# Check columns to see which one to use as a key
print("Columnas en df_actual:", df_actual.columns)
print("Columnas en df_mood:", df_mood.columns)


In [ ]:
# Create DataFrame with only track_id and DURATION
df_duracion = df_mood[['track_id', 'DURATION']].copy()

# Ensure that both track_id are of the same type
df_actual['track_id'] = df_actual['track_id'].astype(int)
df_duracion['track_id'] = df_duracion['track_id'].astype(int)

# Merge
df_final = pd.merge(df_actual, df_duracion, on='track_id', how='left')

# Save results
df_final.to_csv(r"C:\Users\PORTATIL\Desktop\mtg_data\df_con_duration.csv", index=False)

print("Final file saved with the 'DURATION' column added")


In [ ]:
df_final

Finally, we have the complete dataset with all the previous columns (features + mood, instruments and genre tags) along with duration, which will allow us to scale our variables.